In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt  # type:ignore
folder = "genetic_engine_v0.7_41d9335284a2cca22b872397bd109a196ebf4496"

In [ ]:
# 30 minutes
data = pd.read_parquet(f"data/{folder}/synthetic_summary.parquet")
# print(data.columns)
print("Size data:", len(data))
print("Average number of generations:", (len(data)/ 45000))

unique_benchmarks = len(data["Benchmark Name"].unique())
print("Unique benchmarks:", unique_benchmarks)
final_data = list()
not_found = list()
for idx, name in enumerate(data["Benchmark Name"].unique()):
    print(f"{round((idx/unique_benchmarks) * 100,3)} %", end="\r")
    for representation in ["GrammaticalEvolutionRepresentation", "DynamicStructuredGrammaticalEvolutionRepresentation", "TreeBasedRepresentation"]:
        temp_data = data[(data["Benchmark Name"] == name) & (data["Representation"] == representation)]
        # temp_data = data.query(f"'Benchmark Name' == @name & Representation == @representation")
        try:
            # print(temp_data)
            max_id = temp_data["Generations"].idxmax()
            final_data.append(temp_data.loc[max_id])
        except:
            not_found.append((name, representation))
print(f"Number not found: \n {len(not_found)} (/{3 * unique_benchmarks})")
print("Not found: \n ", not_found)
final_data = pd.DataFrame(final_data, columns=data.columns)

final_data.to_csv(f"data/{folder}/final_evolution.csv")
print("Size final data:", len(final_data))
bla = final_data.head(20)
bla


In [ ]:
# When a run reaches a fitness of 0, the GP programme stops. Therefore the last value in the process is the value for that number of the generation. It stops to show in the plots, which is problematic.
# 4 minutes.
data = pd.read_parquet(f"data/{folder}/synthetic_summary.parquet")
print(data.columns)
fitness_difficulties = ["easy", "medium", "hard"]
zeros = data[data["Fitness component 0"] == 0]
print(len(zeros))

lost_values = list()
for idx, zero in zeros.iterrows():
    generation = zero['Generations']
    for i in range(100-generation):
        zero.loc['Generations'] = i + generation + 1
        lost_values.append(zero.copy())

lost_values = pd.DataFrame(lost_values, columns=data.columns)
data = pd.concat([data, lost_values])
data.to_parquet(f"data/{folder}/full_generations.parquet")


In [ ]:
# Meta data is not complete
metadata = pd.read_csv(f"data/{folder}/synthetic_metadata.csv")
# print(metadata.columns)
unique_seeds = metadata.seed.unique()
print(len(unique_seeds))
print(len(metadata)/len(unique_seeds))
print(len(metadata["maximum depth"].unique()))
print(len(metadata["fitness difficulty"].unique()))